**Group Members:**  
21k-3458, Asad  
21k-3169, Anas Imran  
21k-4658, Zehra Fatima

#EMOTIONAL ANALYSIS USING TEXT

In [ ]:
import urllib.request
import zipfile
import os

from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense
import nltk

from nltk.tokenize import word_tokenize
from keras.preprocessing.sequence import pad_sequences
import warnings
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import re


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
nltk.download('punkt')
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

fileopen = open('drive/MyDrive/AIProjectFiles/train.txt','r') # Open the train file
x_train_data = [] # Array for train data
y_train_data = [] # Array for train labels

#is feeling bad  just got bad news about my Mum##worry

for x in fileopen: # Read train file
    sp = x.split('##')  # Split the emotions and text
    y_train_data.append(sp[1].strip())
    x_train_data.append(sp[0])

fileopen = open('drive/MyDrive/AIProjectFiles/test.txt','r') # Open test file
x_test_data = [] # Array for test data
y_test_data = [] # Array for test labels

for x in fileopen: # Read test file
    sp = x.split('##') # Split the emotion and its respective text
    y_test_data.append(sp[1].strip())
    x_test_data.append(sp[0])

fileopen = open('drive/MyDrive/AIProjectFiles/val.txt','r') # Open validation file

for x in fileopen: # Read validation file
    sp = x.split('##') # Split the data separating the emotions and its respective text
    y_test_data.append(sp[1].strip())
    x_test_data.append(sp[0])

# Create DataFrames for train and test data
data_train_DF = pd.DataFrame({'Text': x_train_data, 'Emotion': y_train_data})
# print(data_train_DF)
data_test_DF = pd.DataFrame({'Text': x_test_data, 'Emotion': y_test_data})
# Concatenate train and test DataFrames
d = pd.concat([data_train_DF, data_test_DF], ignore_index=True)

# Print the concatenated DataFrame
# print(d.shape)
# classes=pd.unique(data_train_DF['Emotion'])
# print(len(classes))
# classes=pd.unique(data_test_DF['Emotion'])
# print(len(classes))
classes=pd.unique(d['Emotion'])
print(len(classes))
classes

In [ ]:
########## CLEANING OF DATA ##################
def cleaning_data(d):
    d = re.sub(r"(#[\d\w\.]+)", '', d) #it subsitutes and irrelevant data with empty space basically cleaning of data
    d = re.sub(r"(@[\d\w\.]+)", '', d) #it subsitutes and irrelevant data with empty space basically cleaning of data
    d = word_tokenize(d) #words are broken down into syllabel and stored in a variable called data
    # print(d)
    return d #cleaned data returned from here

texts_data = [' '.join(cleaning_data(text)) for text in d.Text] #sentences are joined
texts_train = [' '.join(cleaning_data(text)) for text in x_train_data] #cleaned data for training is stored in texts_train
texts_test = [' '.join(cleaning_data(text)) for text in x_test_data] #clean data for testing in texts_test

In [ ]:
tokenizer = Tokenizer() # SPLITTING
tokenizer.fit_on_texts(texts_data)  #This method creates the vocabulary index based on word frequency
sequence_train = tokenizer.texts_to_sequences(texts_train) #transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index
sequence_test = tokenizer.texts_to_sequences(texts_test) #transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index
print(sequence_test)
index_of_words = tokenizer.word_index #CREATES A DICT WITH INDEX NUMBER AND THAT NUMBER WAS PREVIOSULY STORED IN WORD_INDEX
# print(index_of_words)
vocab_size = len(index_of_words)+1 # TOTAL SIZE OF DICTINOARY VOCABULARY

In [ ]:
classes = ['anger', 'boredom', 'empty', 'enthusiasm', 'fun', 'happiness','hate', 'love', 'neutral', 'relief', 'sadness', 'surprise','worry']
print('{',end="")
a = 0
for i in classes:
  print("'"+str(i)+"':"+str(a)+",", end="")
  a = a+1
print('}')

In [ ]:
total_classes = len(classes)
print(total_classes)
embed_num_dims = 300
seq_len = 500
class_names = ['anger', 'boredom', 'empty', 'enthusiasm', 'fun', 'happiness','hate', 'love', 'neutral', 'relief', 'sadness', 'surprise','worry']
X_train_pad = pad_sequences(sequence_train,maxlen=seq_len)  #CREATED A 2D ARRAY FIRST AG:sequence of integer that were previoulsy converted into words for training and second coresponds to max lenght
X_test_pad = pad_sequences(sequence_test,maxlen=seq_len) #CREATED A 2D ARRAY FIRST AG:sequence of integer that were previoulsy converted into words for testing and second coresponds to max lenght
encoding_data = {'anger':0,'boredom':1,'empty':2,'enthusiasm':3,'fun':4,'happiness':5,'hate':6,'love':7,'neutral':8,'relief':9,'sadness':10,'surprise':11,'worry':12}
y_train = [encoding_data[x] for x in data_train_DF.Emotion] # All training string gets converted into integer
y_test = [encoding_data[x] for x in data_test_DF.Emotion] #All testing strings gets converted into integer
y_train = to_categorical(y_train) #convert the data into matrix of 0s and 1s
y_test = to_categorical(y_test) #convert the data into matrix of 0s and 1s
print(y_train)
print("\n")
print(y_test)

In [ ]:
! gdown --id 1SsrYFruVmyedeRuS7i8PI3SfRn6sOs0f


In [ ]:
def create_matrix(filepath,index,embedding_dim): # Enter data is passed to this function to convert it into zeros and ones
    vocab_size = len(index)+1 #size of the entire file is calculated
    embedding_matrix = np.zeros((vocab_size,embedding_dim)) # matrix is gonna be created with vocabulary size and dimension provided
    with open(filepath) as f:
        for line in f:
            word,*vect = line.split()
            if word in index:
                id = index[word]
                embedding_matrix[id] = np.array(vect,dtype=np.float32)[:embedding_dim]
    return embedding_matrix

# fname = '/content/wiki-news-300d-1M.vec'
fname = 'drive/MyDrive/AIProjectFiles/wiki-news-300d-1M.vec'
embedd_matrix = create_matrix(fname,index_of_words,embed_num_dims)

In [ ]:
with tf.device('/device:GPU:0'):
  #convert each word into a fixed length vector of defined size. The resultant vector is a dense one with having real values instead of just 0's and 1's.
  layer = Embedding(vocab_size, embed_num_dims, input_length = seq_len, weights = [embedd_matrix],trainable = False) #deep learning with keras
  #vocab_size= size of volcabulary that we are embedding
  #embed_num_dims= Dimension of dense embedding
  #input_lenght=max_seq_len
  #Turns positive integer into fixed size vectors
  output_size = 128
  bidirectional = True
  model = Sequential() # PROVIDES FEATURES ON SEQUENTIAL LAYERS OF STACK
  model.add(layer)
  model.add(Bidirectional(GRU(units = output_size,dropout = 0.2,recurrent_dropout = 0.2)))
  model.add(Dense(total_classes, activation = 'softmax'))
  model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
print(model)


In [ ]:
batch_size = 128
epochs = 8

with tf.device('/device:GPU:0'):
  print(X_train_pad.shape)
  print(y_train.shape)
  print(X_test_pad.shape)
  print(y_test.shape)
  hist = model.fit(X_train_pad, y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_test_pad, y_test))

# X_train = Input data
# Y_train = Test data
# batch_size =Number of samples per gradient
# epochs = An epoch is an iteration over the entire x and y data provided in this case 8 times
# validation_data = Data on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data.


In [ ]:
msg = ["I am sad"]
s = tokenizer.texts_to_sequences(msg) #So it basically takes each word in the text and replaces it with its corresponding integer value
padded = pad_sequences(s,maxlen=seq_len)#CREATED A 2D ARRAY FIRST AG:sequence of integer that were previoulsy converted into words for testing and second coresponds to max lenght
p = model.predict(padded)# It helps you to  returns the labels of the data passed as argument based upon the learned or trained data obtained from the model.

print('Message:' + str(msg))
print('Emotion:', class_names[np.argmax(p)]) # PRINT THE EMOTIONS

In [ ]:
import random

# Sample data for generating emotions
data = [
    ("I am happy.", "joy"),
    ("This is sad.", "sadness"),
    ("I feel afraid.", "fear"),
    ("She loves him.", "love"),
    ("What a surprise!", "surprise"),
    ("I am angry.", "anger")
]

# Generating more data items
generated_data = []
for _ in range(1000):
    sentence = random.choice(["I am", "This is", "I feel", "She", "What a"]) + " " + \
               random.choice(["happy.", "sad.", "afraid.", "loves him.", "surprise!", "angry."])
    emotion = random.choice(["joy", "sadness", "fear", "love", "surprise", "anger"])
    generated_data.append((sentence, emotion))

# Splitting the generated data into train, test, and val sets
train_data = generated_data[:700]
test_data = generated_data[700:850]
val_data = generated_data[850:]

# Writing the data to files
def write_to_file(file_name, data):
    with open(file_name, 'w') as f:
        for item in data:
            f.write(item[0] + ";" + item[1] + "\n")

write_to_file('train.txt', train_data)
write_to_file('test.txt', test_data)
write_to_file('val.txt', val_data)


In [ ]:
import random

# Emotions
emotions = ['anger', 'sadness', 'fear', 'joy', 'surprise', 'love']

# Function to generate random data
def generate_data(num_samples):
    data = []
    for _ in range(num_samples):
        text = ' '.join(['word' + str(random.randint(1, 100)) for _ in range(random.randint(5, 15))])
        emotion = random.choice(emotions)
        data.append((text, emotion))
    return data

# Generate data for train, test, and val sets
train_data = generate_data(1000)
test_data = generate_data(1000)
val_data = generate_data(1000)

# Function to save data to file
def save_data(filename, data):
    with open(filename, 'w') as file:
        for item in data:
            file.write(f"{item[0]};{item[1]}\n")

# Save data to files
save_data('train.txt', train_data)
save_data('test.txt', test_data)
save_data('val.txt', val_data)

print("Data generated and saved successfully!")
